In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np 

import create_model
import pickle
import random
import data_formatting

def encodeSent(sent):

    if type(sent) == str: sent = sent.split(' ')
    
    return [vocab_dict[word] if word in vocab_dict else 2 for word in sent]

def decodeSent(sent):
    return [inv_map[i] for i in sent]

def validate(op, feed_dict):

    for i, (e_in, dt_targ, dt_pred) in enumerate(zip(feed_dict['encoder_inputs:0'], 
                                                 feed_dict['decoder_targets:0'], 
                                                 session.run(op, feed_dict))):

        print('  sample {}:'.format(i + 1))
        #print('    enc input           > {}'.format(e_in))
        print('    enc input           > {}'.format(' '.join([inv_map[i] for i in e_in if i!=0])))

        #print('    dec input           > {}'.format(dt_targ))
        print('    dec input           > {}'.format(' '.join([inv_map[i] for i in dt_targ if i!=0])))

        #print('    dec train predicted > {}'.format(dt_pred))
        print('    dec train predicted > {}'.format(' '.join([inv_map[i] for i in dt_pred if i!=0])))

        if i >= 0: break

tf.reset_default_graph()

In [2]:
vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl', 'rb'))
inv_map = {v: k for k, v in vocab_dict.items()}

In [3]:
#dataset = 'twitter'

#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl')

#df_all['alpha_Pair_1_encoding'] =  df_all['alpha_Pair_1_tokens'].apply(encodeSent)
#df_all['alpha_Pair_0_encoding'] = df_all['alpha_Pair_0_tokens'].apply(encodeSent)

#df_all['Index'] = df_all.index.values

#df_all_train = df_all.sample(frac=0.90, random_state=0)

#df_all_dev = df_all[df_all['Index'].isin(df_all_train['Index'].values) == False]

#df_all_test = df_all_dev.sample(frac=0.10, random_state=0)

#df_all_dev = df_all_dev[df_all_dev['Index'].isin(df_all_test['Index'].values) == False]

In [4]:
#df_all_train[['alpha_Pair_0_encoding', 'alpha_Pair_1_encoding']].to_pickle('../processed_data/queue_testing.pkl')

In [5]:
model_params = {'n_cells':128, 'num_layers':2, 'embedding_size':1024, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':8,
         # 'vocab_size':20 + 1,           
          'beam_width':10, 'encoder_output_keep':0.95, 'decoder_output_keep':0.95, 'seq_low':4, 'seq_high':15
         }

In [6]:
training_params = { 'vocab_lower':3, 'vocab_upper':model_params['vocab_size']-1, 
                    'n_epochs':1000, 'batches_in_epoch':1000}
                   #'batches_in_epoch':int(df_all_train.shape[0]/model_params['minibatch_size'])}

In [7]:
tf.reset_default_graph()

train_model = create_model.Model(model_params, 'train')

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.01
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, \
                                   training_params['n_epochs']*training_params['batches_in_epoch'], 0.0001, staircase=False)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(train_model.loss, global_step=global_step)

In [8]:
entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=train_model.decoder_train_targets,
    logits=train_model.decoder_outputs_train.rnn_output
)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

sess.run(train_model.decoder_train_length)

In [ ]:
        self.pad_size = self.decoder_train_length - tf.shape(self.decoder_outputs_train.rnn_output)[1]
        
        #self.decoder_outputs_train_logit = tf.pad(self.decoder_outputs_train.rnn_output, [[0, 0], [0,  self.pad_size], [0, 0]])

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

sess.run(train_model.pad_size)

In [ ]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()

    coord = tf.train.Coordinator()
   
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(100000):
        
        session.run(train_op)
        if i%100 == 0: 
            print ('i:', i)
            print(session.run(train_model.loss))

          #  saver.save(session, 'chkpt/seq2seq_twitter_test_queue', global_step = tf.train.global_step(session, global_step))

           # print ('Session saved')
        
    coord.request_stop()
    # ... and we wait for them to do so before releasing the main thread
    coord.join(threads)

i: 0
10.1939
i: 100
6.20184
i: 200
5.53671
i: 300
5.32172


In [11]:
df_all_train = pd.read_pickle('../processed_data/queue_testing.pkl')
df_all_train = df_all_train[:5000]
training_data = data_formatting.prepare_train_batch(df_all_train['alpha_Pair_0_encoding'].values, 
                                                    df_all_train['alpha_Pair_1_encoding'].values)

In [23]:
training_data[2][16:24]

array([[23459, 22583,  7202,  6787, 24568, 24207,     1,     0,     0,
            0,     0,     0,     0,     0,     0],
       [26857,  5534,  2197,  1308, 25137, 12609, 24336,  8621, 13584,
        19775,     1,     0,     0,     0,     0],
       [13250, 26711,  3900, 14422,  9175,   562, 22595, 17636,     1,
            0,     0,     0,     0,     0,     0],
       [   21,  8333, 10409,  8587, 19570, 12684,     1,     0,     0,
            0,     0,     0,     0,     0,     0],
       [24413,  6918,  3836, 20716,  2762, 24336,  1068, 10675, 12983,
         7616,  6618,     1,     0,     0,     0],
       [  426, 11240,   233, 12609,  4512, 10600, 17237, 13374,     1,
            0,     0,     0,     0,     0,     0],
       [26331, 11417, 23426, 25276, 14982, 25934, 13694,  6093,  3060,
            1,     0,     0,     0,     0,     0],
       [23968, 19039, 16994, 10675, 13291, 24336, 14675, 19039,  1350,
        13178, 21875,     1,     0,     0,     0]])

In [33]:
results_1[5]

16

In [36]:
results_1[0].shape, results_1[1].shape, results_1[4].shape

((8, 16), (8, 13, 27132), (8, 16, 27132))

In [23]:
tf.shape(tf.pad(results_1[1], [[0, 0], [0, 5], [0, 0]], "CONSTANT"))[1]

<tf.Tensor 'strided_slice_2:0' shape=() dtype=int32>

In [19]:
results_1[1][0][0]

array([ 0.01233576, -0.00258708, -0.01116863, ..., -0.00037263,
       -0.00256451, -0.00534932], dtype=float32)

In [14]:
results_1[3]

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
         0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,
         0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         0.,  0.,  0.]], dtype=float32)

In [126]:
len(results_1[1][0])

13

In [46]:
len(decoder_outputs_train[0])

16

In [32]:
trainable_parameters = np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])

In [34]:
tf.trainable_variables()

[<tf.Variable 'embedding_matrix:0' shape=(27132, 1024) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0' shape=(1152, 512) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/lstm_cell/kernel:0' shape=(256, 512) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'memory_layer/kernel:0' shape=(128, 128) dtype=float32_ref>,
 <tf.Variable 'decoder/multi_rnn_cell/cell_0/lstm_cell/kernel:0' shape=(1152, 512) dtype=float32_ref>,
 <tf.Variable 'decoder/multi_rnn_cell/cell_0/lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'decoder/multi_rnn_cell/cell_1/Attention_Wrapper/lstm_cell/kernel:0' shape=(384, 512) dtype=float32_ref>,
 <tf.Variable 'decoder/multi_rnn_cell/cell_1/Attention_Wrapper/lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'decoder/multi_rnn_cel

In [12]:
trainable_parameters

11234428

In [ ]:
df_all_train.shape[0]/trainable_parameters

0.0086031082312334906

In [ ]:
print_interval = 1
save_interval = 3

train_loss = []
dev_loss = []
learning_rate = []

n_batch_size  = training_params['minibatch_size']

init = tf.global_variables_initializer()

with tf.Session() as session:
    
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    #saver.restore(session, 'chkpt/seq2seq_twitter_sample-12880')

    for epoch in range(training_params['n_epochs']):
        
        df_all_train = df_all_train.sample(frac=1, random_state=tf.train.global_step(session, global_step))
       
        encoded_text = df_all_train['alpha_Pair_0_encoding'].values
        decoded_text = df_all_train['alpha_Pair_1_encoding'].values
        
        text_index = df_all_train['Index'].values
        
        input_batches = ([encoded_text[block_idx*n_batch_size:(block_idx+1)*n_batch_size], 
                 decoded_text[block_idx*n_batch_size:(block_idx+1)*n_batch_size], 
                         text_index[block_idx*n_batch_size:(block_idx+1)*n_batch_size]]\
                    for block_idx in range(len(encoded_text)))
        
        for batch in range(training_params['batches_in_epoch']):
            
            if batch % 100 == 0: print (batch)
            
            epoch_batches = next(input_batches)

            input_batch_data = epoch_batches[0]
            target_batch_data = epoch_batches[1]
            batch_data_index = epoch_batches[2]

            fd = data_formatting.prepare_train_batch(input_batch_data, target_batch_data)

            feed_dict = {'encoder_inputs:0': fd[0],
                         'encoder_inputs_length:0': fd[1],
                         'decoder_targets:0': fd[2],
                         'decoder_targets_length:0': fd[3]}

            results = session.run(train_op, feed_dict)

        print ('epoch:%d, global_step:%s, learning rate:%.3g' % 
                   (epoch, tf.train.global_step(session, global_step), session.run(optimizer._lr)))
        
        train_minibatch_loss = session.run(train_model.loss, feed_dict)
        
        train_loss.append([tf.train.global_step(session, global_step),train_minibatch_loss])  
        
        print ('training minibatch loss:%.6g' % (train_minibatch_loss))

        df_sample = df_all_dev.sample(n=n_batch_size, random_state=tf.train.global_step(session, global_step))

        input_batch_data = df_sample['alpha_Pair_0_encoding'].values
        target_batch_data = df_sample['alpha_Pair_1_encoding'].values

        fd_dev = data_formatting.prepare_train_batch(input_batch_data, target_batch_data)

        feed_dict_dev = {'encoder_inputs:0': fd_dev[0],
                         'encoder_inputs_length:0': fd_dev[1],
                         'decoder_targets:0': fd_dev[2],
                         'decoder_targets_length:0': fd_dev[3]}

        dev_minibatch_loss = session.run(train_model.loss, feed_dict_dev)

        validate(train_model.decoder_pred_train, feed_dict_dev) 

        print ('dev minibatch loss:%.6g' % (dev_minibatch_loss))

        dev_loss.append([tf.train.global_step(session, global_step), dev_minibatch_loss])
        
        learning_rate.append([tf.train.global_step(session, global_step), session.run(optimizer._lr)])
        
        if epoch % save_interval == 0: 

            saver.save(session, 'chkpt/seq2seq_twitter_sample', global_step = tf.train.global_step(session, global_step))

            print ('Session saved')

0
100
200
300
400
500
600
700
epoch:0, global_step:755, learning rate:0.00991
training minibatch loss:5.21919
  sample 1:
    enc input           > who would ve known life work in crazy way <EOS>
    dec input           > omg what a throwback to think our daughter were born on the same day <EOS>
    dec train predicted > i i s a to the <EOS> first <EOS> in to the debate <EOS> <EOS>
dev minibatch loss:4.92515
Session saved
0
100
200
300
400
500
600
700
epoch:1, global_step:1510, learning rate:0.00982
training minibatch loss:4.93339
  sample 1:
    enc input           > that s why my mom doe it for me <EOS>
    dec input           > i hate food shopping because it s always so expensive <EOS>
    dec train predicted > i m the <EOS> <EOS> i s a <EOS> cute <EOS>
dev minibatch loss:4.96667
0
100
200
300
400
500
600
700
epoch:2, global_step:2265, learning rate:0.00973
training minibatch loss:4.67621
  sample 1:
    enc input           > true and it make you stronger <EOS>
    dec input       

100
200
300
400
500
600
700
epoch:23, global_step:18120, learning rate:0.00802
training minibatch loss:4.13489
  sample 1:
    enc input           > doe trump need to blow his nose <EOS>
    dec input           > she threw first punch <EOS>
    dec train predicted > i s a to to
dev minibatch loss:5.11523
0
100
200
300
400
500
600
700
epoch:24, global_step:18875, learning rate:0.00794
training minibatch loss:4.1465
  sample 1:
    enc input           > did you enjoy it though <EOS>
    dec input           > im so sad it over i never wanted it to end <EOS>
    dec train predicted > i so excited <EOS> s <EOS> m want to <EOS> you <EOS>
dev minibatch loss:5.38686
Session saved
0
100
200
300
400
500
600
700
epoch:25, global_step:19630, learning rate:0.00787
training minibatch loss:4.09137
  sample 1:
    enc input           > section row seat <EOS>
    dec input           > went deep now to enter to win a <EOS>
    dec train predicted > went deep captain for enter to win a chance
dev minibat

  sample 1:
    enc input           > oh my god i bet they made bracelet out of dental floss <EOS>
    dec input           > stahp i bet they made matching friendship lanyard <EOS>
    dec train predicted > i s m i are me <EOS> <EOS> <EOS>
dev minibatch loss:5.58421
0
100
200
300
400
500
600
700
epoch:47, global_step:36240, learning rate:0.00643
training minibatch loss:3.9788
  sample 1:
    enc input           > i should be so lucky thats a winning personality right there <EOS>
    dec input           > careful and they are watching <EOS>
    dec train predicted > i i i re going the
dev minibatch loss:5.11879
0
100
200
300
400
500
600
700
epoch:48, global_step:36995, learning rate:0.00637
training minibatch loss:3.84454
  sample 1:
    enc input           > this is such a weird show love it <EOS>
    dec input           > and now for some to hopefully take my mind off it <EOS>
    dec train predicted > i i this the dedicated the this <EOS> vest <EOS> <EOS> s
dev minibatch loss:5.77557

  sample 1:
    enc input           > deleting of thousand of email from her private server wasnt hiding evidence and the <EOS>
    dec input           > a long a you re not a republican patriot anti hillary obama <EOS>
    dec train predicted > i a a a <EOS> a even racist <EOS> <EOS> <EOS> <EOS> <EOS>
dev minibatch loss:5.49104
Session saved
0
100
200
300
400
500
600
700
epoch:70, global_step:53605, learning rate:0.0052
training minibatch loss:3.70549
  sample 1:
    enc input           > i guess my joke isnt funny anymore if it ever wa <EOS>
    dec input           > gmail support medium query a of yesterday <EOS>
    dec train predicted > i the the <EOS> <EOS> a course <EOS>
dev minibatch loss:5.68992
0
100
200
300
400
500
600
700
epoch:71, global_step:54360, learning rate:0.00515
training minibatch loss:3.86835
  sample 1:
    enc input           > cry you still at scammer bitch where my green pant ? <EOS>
    dec input           > i wa a scammer for year but retired <EOS>
    dec 

Session saved
0
100
200
300
400
500
600
700
epoch:91, global_step:69460, learning rate:0.00429
training minibatch loss:3.72008
  sample 1:
    enc input           > will try next time <EOS>
    dec input           > i ve had that problem usually just killing the app store app work <EOS>
    dec train predicted > i m been to passion <EOS> <EOS> sliding me stage <EOS> <EOS> <EOS> <EOS>
dev minibatch loss:5.87981
0
100
200
300
400
500
600
700
epoch:92, global_step:70215, learning rate:0.00425
training minibatch loss:3.69645
  sample 1:
    enc input           > good point kerry concedes <EOS>
    dec input           > if boeing had never developed the then would not have been shot down <EOS>
    dec train predicted > trump you with a fails a outrage <EOS> be be done involved <EOS> <EOS>
dev minibatch loss:5.81734
0
100
200
300
400
500
600
700
epoch:93, global_step:70970, learning rate:0.00421
training minibatch loss:3.6244
  sample 1:
    enc input           > tell that to hillary a we do

  sample 1:
    enc input           > last i looked bill clinton isn t running for potus <EOS>
    dec input           > defend her now poor liberal cant win <EOS>
    dec train predicted > i the asset on millennial pepper be the
dev minibatch loss:5.87594
0
100
200
300
400
500
600
700
epoch:114, global_step:86825, learning rate:0.00347
training minibatch loss:3.73476
  sample 1:
    enc input           > we are expert in service in our portfolio email to leadscom <EOS>
    dec input           > i m still developing it any website designer out there ? <EOS>
    dec train predicted > any need looking looking any s more design <EOS> <EOS> <EOS> <EOS>
dev minibatch loss:5.79397
Session saved
0
100
200
300
400
500
600
700
epoch:115, global_step:87580, learning rate:0.00344
training minibatch loss:3.76171
  sample 1:
    enc input           > yeah she wa assigned gr <EOS>
    dec input           > omg i wa gon na say did sam choose him ? oh thank god <EOS>
    dec train predicted > she she 

Session saved
0
100
200
300
400
500
600
700
epoch:136, global_step:103435, learning rate:0.00283
training minibatch loss:3.55833
  sample 1:
    enc input           > i ll miss everyone around the office <EOS>
    dec input           > good luck we ll miss you around the office <EOS>
    dec train predicted > i morning i are be you <EOS> <EOS> jungle <EOS>
dev minibatch loss:5.81347
0
100
200
300
400
500
600
700
epoch:137, global_step:104190, learning rate:0.00281
training minibatch loss:3.59487
  sample 1:
    enc input           > folk who work on open source still got ta pay the bill somehow <EOS>
    dec input           > just feel like a great open sourced project for motivated people <EOS>
    dec train predicted > the ranking like a homophobe question university monitoring among the to are
dev minibatch loss:6.15257
0
100
200
300
400
500
600
700
epoch:138, global_step:104945, learning rate:0.00278
training minibatch loss:3.71126
  sample 1:
    enc input           > you b knowin

100
200
300
400
500
600
700
epoch:159, global_step:120800, learning rate:0.00229
training minibatch loss:3.47863
  sample 1:
    enc input           > listen to this and you can get those player you need <EOS>
    dec input           > yes sir must listen pre tuesday <EOS>
    dec train predicted > i i i be to thing <EOS>
dev minibatch loss:5.61197
Session saved
0
100
200
300
400
500
600
700
epoch:160, global_step:121555, learning rate:0.00227
training minibatch loss:3.52837
  sample 1:
    enc input           > bih it s homework <EOS>
    dec input           > all i see on my snapchat is the trump and hillary thing <EOS>
    dec train predicted > i of want this tumblr tl is bonnie best dab receiving <EOS> <EOS>
dev minibatch loss:5.79507
0
100
200
300
400
500
600
700
epoch:161, global_step:122310, learning rate:0.00225
training minibatch loss:3.56731
  sample 1:
    enc input           > it abt the sound not the name but signed producer get salary tho <EOS>
    dec input           > i

Session saved
0
100
200
300
400
500
600
700
epoch:181, global_step:137410, learning rate:0.00187
training minibatch loss:3.55369
  sample 1:
    enc input           > he doe that a a joke he been wearing them for a minute <EOS>
    dec input           > i wa just looking at this <EOS>
    dec train predicted > pope m blinded a for cp <EOS>
dev minibatch loss:5.93854
0
100
200
300
400
500
600
700
epoch:182, global_step:138165, learning rate:0.00185
training minibatch loss:3.56465
  sample 1:
    enc input           > legitimate reason to have kid <EOS>
    dec input           > i can t wait to have kid and eat all their leftover <EOS>
    dec train predicted > i m t be to kfc worn <EOS> i <EOS> of <EOS> <EOS>
dev minibatch loss:6.0006
0
100
200
300
400
500
600
700
epoch:183, global_step:138920, learning rate:0.00184
training minibatch loss:3.50842
  sample 1:
    enc input           > tell him he ll be aight <EOS>
    dec input           > he told me to tell u to stop taking screen shot

In [16]:
tf.reset_default_graph()

In [17]:
inf_model = create_model.Model(model_params, 'infer')

In [19]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as session:
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(session, 'chkpt/seq2seq_twitter_sample-30200')
    
    #ran_seq = generateRandomSeqBatchMajor(length_from=training_params['seq_low'], length_to=training_params['seq_high'],
    #                                       vocab_lower=2, vocab_upper=training_params['vocab_upper'],
    #                                       batch_size=training_params['minibatch_size'])
    
    #input_batch_data = ran_seq
    
    df_sample = df_all_dev.sample(n=1, random_state=0)
#    df_sample = df_all_train.sample(n=1)

    input_batch_data = df_sample['alpha_Pair_0_encoding'].values
    target_batch_data = df_sample['alpha_Pair_1_encoding'].values
    
    fd = data_formatting.prepare_train_batch(input_batch_data, target_batch_data)

    fd_inf = data_formatting.prepare_batch(input_batch_data)
    
    feed_dict_inf = {'encoder_inputs:0': fd_inf[0],
                     'encoder_inputs_length:0': fd_inf[1]}
    
    #inf_out = session.run([inf_model.decoder_pred_decode, inf_model.decoder_pred_decode_prob], feed_dict_inf)
    inf_out = session.run([inf_model.decoder_pred_decode], feed_dict_inf)
    
    for (e_in, dt_in, dt_inf) in zip(fd[0], fd[2], inf_out[0]):
        print ('e_in', decodeSent(e_in))
        print ('d_in', decodeSent(dt_in))        
        #print ('_inf', decodeSent(dt_inf))
        print (' ')

INFO:tensorflow:Restoring parameters from chkpt/seq2seq_twitter_sample-30200
e_in ['no', 'he', 'wa', 'just', 'some', 'nigger', 'that', 'obama', 'used', 'to', 'hang', 'with', '<EOS>']
d_in ['obama', 'hired', 'van', 'jones', 'too', 'wasn', 't', 'he', 'a', 'communist', '?', '<EOS>']
 


In [ ]:
#WHY DOES BEAM SEARCH PRODUCE A -1?

In [24]:
inv_map[-1] = 'NULL'

In [25]:
[inv_map[i] for i in list(zip(*inf_out[0][0]))[0]]

['trump',
 's',
 'lack',
 'of',
 'civility',
 '<EOS>',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL']

In [20]:
with tf.Session() as session:
    #session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(session, 'chkpt/seq2seq_twitter_sample-236096')
    inf_out = session.run([inf_model.decoder_outputs_decode], 
                          feed_dict_inf)
    

INFO:tensorflow:Restoring parameters from chkpt/seq2seq_twitter_sample-236096


In [40]:
decodeSent(feed_dict_inf['encoder_inputs:0'][0])

['have',
 'you',
 'been',
 'toggling',
 'off',
 'the',
 'bluetooth',
 'radio',
 'or',
 'is',
 'it',
 'always',
 'on',
 '?',
 '<EOS>']

In [53]:
decodeSent(list(zip(*q.beam_search_decoder_output.predicted_ids[0]))[0])

['my',
 'bose',
 'headphone',
 'are',
 'so',
 'constantly',
 'disconnecting',
 'randomly',
 'from',
 'my',
 'iphone',
 'but',
 'not',
 'any',
 'other',
 'device',
 '<EOS>']

In [44]:
inv_map[13983]

'my'

In [45]:
q.beam_search_decoder_output.predicted_ids[0]

array([[13983, 10486,  1006,  7185, 16110, 11633],
       [ 7881,  7881,  7881, 14596,  7881, 16584],
       [ 2915,  2915, 14575, 16584,  4785, 15563],
       [ 6297,  6297,  6297, 13473,  6297,  6297],
       [15464, 14856,  7524,  4569, 16749, 15283],
       [10829, 15464, 11589, 15464,  6063,  7773],
       [ 8320, 10829,  7524, 10673,  9781,  5053],
       [15563,  8320,     1, 12302,  4378, 13983],
       [16749, 15563,     1, 13983,  3104,  6824],
       [13983, 16749,  6385,     1,  2175,  8748],
       [16584, 13983,  1655,     1,  1972, 11172],
       [ 1655, 11172, 16584, 13983,     1, 10486],
       [11773,     1,  1655,  7881,     1,  2175],
       [ 3104,     1, 11773,  2915,     1, 14215],
       [ 2175,     1,  1492,  3104, 16451,     1],
       [ 7218,     1,     1,  2175, 16451,     1],
       [    1,     1,     1,     1,  7218,     1]])

In [47]:
decodeSent(q.beam_search_decoder_output.predicted_ids[0][0])

['my', 'the', 'your', 'it', 'their', 'this']

In [56]:
list(zip(*q.beam_search_decoder_output.scores[0]))

[(-0.035850089,
  -0.035851996,
  -0.038993467,
  -0.039482821,
  -0.35288915,
  -0.3532013,
  -0.35320261,
  -0.35321036,
  -0.35325852,
  -0.35838923,
  -0.36694854,
  -0.67802882,
  -0.68024153,
  -0.68467307,
  -0.68513405,
  -0.69088089,
  -0.69419718),
 (-3.3503947,
  -3.3508077,
  -3.5386569,
  -3.5386579,
  -1.4511024,
  -1.7200738,
  -1.7201383,
  -1.7316133,
  -1.7316157,
  -2.0672543,
  -2.0672672,
  -1.7039001,
  -1.7039022,
  -1.7039022,
  -1.7039022,
  -1.7039022,
  -1.7039022),
 (-8.9498634,
  -8.9612808,
  -5.4555082,
  -5.8360171,
  -3.8721921,
  -3.3295441,
  -3.6099124,
  -4.0482016,
  -4.0482016,
  -3.0341046,
  -3.0658193,
  -2.0685022,
  -2.0700371,
  -2.1878219,
  -2.651706,
  -2.883266,
  -2.883266),
 (-11.533058,
  -11.431015,
  -6.8540144,
  -6.8631053,
  -3.9936094,
  -3.8760388,
  -3.9421754,
  -4.2694464,
  -4.6147184,
  -4.0482016,
  -4.0482016,
  -3.2055624,
  -3.2584443,
  -3.288914,
  -3.2263942,
  -3.2267132,
  -3.3478928),
 (-13.323359,
  -11.533113,


In [31]:
q.beam_search_decoder_output.scores[0][1]

array([ -0.035852  ,  -3.35080767,  -8.96128082, -11.43101501,
       -11.53311253, -12.55368805], dtype=float32)

In [104]:
inf_model.inference_decoder.step

<bound method BeamSearchDecoder.step of <tensorflow.contrib.seq2seq.python.ops.beam_search_decoder.BeamSearchDecoder object at 0x000002A304336400>>

In [45]:
t = inf_out[0]

In [40]:
len(inf_out[0][0]) #hidden states and cell states of single LSTM

2

In [41]:
len(inf_out[0][0][0]) #tile duplicates of the LSTM equal to beam_width

3

In [43]:
len(inf_out[0][0][0][0]) #number of cells

128